In [80]:
import requests
import json
 
subreddit = 'wallstreetbets'
limit = 10
timeframe = 'year' #hour, day, week, month, year, all
listing = 'all' # controversial, best, hot, new, random, rising, top
 
def get_reddit(subreddit,listing,limit,timeframe,after=None):
    try:
        
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        #base_url = f'https://www.reddit.com/r/{subreddit}.json?limit={limit}&t={timeframe}'
        if after != None:
            base_url = f'{base_url}&after={after}'
            
            print(f'After Url: ' + base_url)
        request = requests.get(base_url, headers = {'User-agent': '/u/awesome-blossom'})
    except:
        print('An Error Occured')
    return request.json()
 
#r = get_reddit(subreddit,listing,limit,timeframe)

In [74]:
#//*** Introduces a delay with a random component
#//*** base_wait_time = The minimum wait time in seconds (default 1 second)
#//*** random_interval = used to generate a random number of milliseconds to delay. 100 = between 0 and 1 second random delay
def random_delay(base_wait_time=1,random_interval=100):
    import time
    import random
    loop_delay = (random.randrange(random_interval)/100)+base_wait_time
    print(f"Waiting: {loop_delay}s")
    time.sleep(loop_delay)

In [78]:
import pandas as pd
import time
import random
#//*** The Keys are the dataframe columns, the value is the subreddit JSON field.
#//*** This value determines the data stored and collected. Everything is built from subfields
subfields = {
    'date' : 'created_utc',
    'up' : 'ups',
    'comments' : 'num_comments',
    'down' : 'downs',
    'cross' : 'num_crossposts',
    'link' : 'permalink', 
    'title' : 'title'
}

#//*** Initialize the subreddit DataFrame
sub_df = pd.DataFrame(columns=subfields.keys())



break_counter = 0

#//*** Perform first download
r = get_reddit(subreddit,listing,limit,timeframe)

for loop_counter in range(9):
    print("Loop Counter", loop_counter)
    #//*** Children are the subreddit posts
    children = r["data"]["children"]
    
    #//*** Process Each Child
    for child in children:

        #//*** Get the sub element
        post = child['data']
        
        #//*** Temp list representing one row
        row_list = []
        
        #//*** Get values based on subfields (Cool right?)
        #//*** Date needs conversion, everything else is a 1:1 add to the row_list
        for key,value in subfields.items():

            #//*** If Date value, convert UTC to datetime
            if key == 'date':
                row_list.append(datetime.fromtimestamp(post[value]))
                continue
            
            #//*** Add all other values to row list
            #//*** Since everything is based on subfields, all elements are in order for the dataframe
            row_list.append(post[value])

        #//*** Add row_list to sub_df
        sub_df = sub_df.append(pd.Series(row_list,index=sub_df.columns),ignore_index=True)

    #//*** All Children Processed
    
    #//*** Check for After value. If NOT None, there are more subbreddits to harvest
    print("After: ", r['data']['after'])
    after = r['data']['after']
    
    #//*** If None, break the loop
    if  after == None:
        break
    
    #//*** Wait between 2 and 3 seconds before the next request
    random_delay(2)
    
    #//*** Make another Requests Call.
    r = get_reddit(subreddit,listing,limit,timeframe,after)

#//*** END Subbreddit Gathering Loop

print(sub_df)

Loop Counter 0


KeyError: 'data'

In [81]:
r = get_reddit(subreddit,listing,limit,timeframe)

print(r)

{'message': 'Not Found', 'error': 404}


In [57]:
len(sub_df['link'].unique())

769

In [59]:
for row in sub_df.iterrows():
    print(row[1]['date']," ", row[1]['comments'], " ", row[1]['up'], " ", row[1]['link'])

2021-06-21 12:17:55   3   3   /r/wallstreetbets/comments/o531p0/can_someone_explain_why_people_are_buying_other/
2021-06-21 12:16:08   1   3   /r/wallstreetbets/comments/o5307c/crsr_a_growth_play_disguised_as_a_value_play/
2021-06-21 12:13:10   2   4   /r/wallstreetbets/comments/o52xr3/why_sofi_ss_an_absolute_steal_of_a_lifetime_right/
2021-06-21 12:13:04   2   3   /r/wallstreetbets/comments/o52xny/ride_introduces_call_of_duty_level_modular_vehicle/
2021-06-21 12:12:10   4   11   /r/wallstreetbets/comments/o52wx1/clne_going_all_the_way_with_clne_see_you_at_50/
2021-06-21 12:08:40   6   40   /r/wallstreetbets/comments/o52u2m/keep_the_wish_push_apes/
2021-06-21 12:07:55   11   0   /r/wallstreetbets/comments/o52tgb/broken_camera_on_brand_new_iphone_reveals_macro/
2021-06-21 12:01:25   8   21   /r/wallstreetbets/comments/o52nzx/push_it/
2021-06-21 11:59:52   16   0   /r/wallstreetbets/comments/o52mfx/boring_stock_to_the_moon_buy_macys/
2021-06-21 11:59:01   5   8   /r/wallstreetbets/commen

2021-06-17 13:01:04   42   124   /r/wallstreetbets/comments/o26bo2/whos_pltrding_with_me/
2021-06-17 13:00:58   51   99   /r/wallstreetbets/comments/o26bkt/grab_your_maps_were_going_to_the_amazon_of_weed/
2021-06-17 13:00:15   13668   667   /r/wallstreetbets/comments/o26ark/what_are_your_moves_tomorrow_june_18_2021/
2021-06-17 12:59:34   24   0   /r/wallstreetbets/comments/o26a2t/stem_is_the_next_10000_bagger_read_this/
2021-06-17 12:59:32   20   43   /r/wallstreetbets/comments/o26a1t/money_printer_go_brrrrrrrrrrr_j_pow_you_a_dove/
2021-06-17 12:54:25   358   896   /r/wallstreetbets/comments/o265mw/tomorrows_quad_witching_day_might_actually_be_the/
2021-06-17 12:53:40   28   39   /r/wallstreetbets/comments/o264zb/prpl_a_dd_i_wrote_on_adderall/
2021-06-17 12:50:30   65   334   /r/wallstreetbets/comments/o2627u/yes_this_ape_still_hasnt_sold_the_dips_everyone/
2021-06-17 12:49:02   40   508   /r/wallstreetbets/comments/o260za/wish_yolo_gimme_my_student_loan_money/
2021-06-17 12:48:00   10

2021-06-15 20:08:38   76   59   /r/wallstreetbets/comments/o0vb17/my_first_major_loss_on_my_first_option_buy/
2021-06-15 20:07:41   72   130   /r/wallstreetbets/comments/o0vagb/viacomcbs_viac_the_only_real_deep_value_left_in/
2021-06-15 20:06:04   87   779   /r/wallstreetbets/comments/o0v9f9/amc_poised_to_take_over_pacific_movie_theaters_at/
2021-06-15 19:58:18   15   0   /r/wallstreetbets/comments/o0v4c3/here_is_a_high_percentage_win_sleep_well_at_night/
2021-06-15 19:54:12   107   614   /r/wallstreetbets/comments/o0v1vi/my_final_wish_yolo/
2021-06-15 19:53:55   90   425   /r/wallstreetbets/comments/o0v1qa/is_blackberry_qnx_a_rotary_phone/
2021-06-15 19:53:08   10   11   /r/wallstreetbets/comments/o0v1ar/this_might_make_the_second_dildo_on_my_graph_34k/
2021-06-15 19:20:34   156   329   /r/wallstreetbets/comments/o0uh9k/bark_12_mm_position_dog_brand_ready_to_takeover/
2021-06-15 19:15:14   73   168   /r/wallstreetbets/comments/o0udtf/now_that_shit_has_somewhat_calmed_down_here_lets/
2

In [50]:
for row in sub_df.iterrows():
    print(row[1]['date']," ", row[1]['comments'], " ", row[1]['up'], " ", row[1]['link'])

2021-06-21 12:06:17   1   1   /r/wallstreetbets/comments/o52s5g/well_wlyyf_well_health_on_the_move/
2021-06-21 12:04:09   3   0   /r/wallstreetbets/comments/o52qby/amc_is_old_news_im_crushing_it_with_knbe/
2021-06-21 12:01:25   6   11   /r/wallstreetbets/comments/o52nzx/push_it/
2021-06-21 11:59:52   9   0   /r/wallstreetbets/comments/o52mfx/boring_stock_to_the_moon_buy_macys/
2021-06-21 11:59:01   3   6   /r/wallstreetbets/comments/o52lqs/amd_already_beyond_the_galaxy_and_google_is/
2021-06-21 11:54:56   2   0   /r/wallstreetbets/comments/o52idk/x_and_the_great_return/
2021-06-21 11:52:45   1   0   /r/wallstreetbets/comments/o52gka/api_quick_analysis/
2021-06-21 11:51:06   2   0   /r/wallstreetbets/comments/o52f7c/61k_colm_yolo/
2021-06-21 11:40:45   7   23   /r/wallstreetbets/comments/o526kn/rest_of_my_portfolio_here_we_go_clne/
2021-06-21 11:38:40   9   8   /r/wallstreetbets/comments/o524ui/giving_bears_prpl_nurples/
2021-06-21 11:34:14   4   0   /r/wallstreetbets/comments/o5216g/na

2021-06-17 13:31:30   64   232   /r/wallstreetbets/comments/o2725l/amd_used_my_lunch_money_for_this_not_bad_for_a/
2021-06-17 13:28:36   45   390   /r/wallstreetbets/comments/o26zm4/bb_retarded_yolo_update_spoiler_alerti_bought_more/
2021-06-17 13:27:41   134   90   /r/wallstreetbets/comments/o26yue/jim_cramer_clashes_with_wallstreetbets_forget/
2021-06-17 13:22:11   17   8   /r/wallstreetbets/comments/o26ug9/steel_dynamics/
2021-06-17 13:21:06   21   101   /r/wallstreetbets/comments/o26tiz/do_you_remember_palantir_220k_yolo_love_cathie/
2021-06-17 13:20:27   16   5   /r/wallstreetbets/comments/o26swj/nvda_has_saved_my_portfolio_lfg/
2021-06-17 13:17:10   104   404   /r/wallstreetbets/comments/o26q07/24000_gainzzz_today_and_113000_total_tendies_so/
2021-06-17 13:13:01   41   51   /r/wallstreetbets/comments/o26ma9/znga_boring_is_not_boring/
2021-06-17 13:07:40   19   47   /r/wallstreetbets/comments/o26hbx/prpl_what_can_i_say_i_like_to_sleep/
2021-06-17 13:07:29   11   11   /r/wallstreet

2021-06-15 19:53:55   90   423   /r/wallstreetbets/comments/o0v1qa/is_blackberry_qnx_a_rotary_phone/
2021-06-15 19:53:08   10   10   /r/wallstreetbets/comments/o0v1ar/this_might_make_the_second_dildo_on_my_graph_34k/
2021-06-15 19:20:34   156   327   /r/wallstreetbets/comments/o0uh9k/bark_12_mm_position_dog_brand_ready_to_takeover/
2021-06-15 19:15:14   73   174   /r/wallstreetbets/comments/o0udtf/now_that_shit_has_somewhat_calmed_down_here_lets/
2021-06-15 19:05:30   55   88   /r/wallstreetbets/comments/o0u7ub/guide_for_tomorrow_aftermath_of_fed_announcement/
2021-06-15 19:03:03   45   196   /r/wallstreetbets/comments/o0u69j/all_in_down_50_wish_lotto_125_june_18_calls/
2021-06-15 18:57:07   49   124   /r/wallstreetbets/comments/o0u2e0/wen_lambo_daddy_cohen/
2021-06-15 18:27:33   163   758   /r/wallstreetbets/comments/o0tk0n/amc_was_good_to_me_today_and_decided_to_bless_me/
2021-06-15 18:22:42   61   221   /r/wallstreetbets/comments/o0th2g/joined_you_today_amc_to_the_moon/
2021-06-15 1

In [11]:
import time 
from datetime import datetime

def process_results(r,input_df):
    print("ModHash: ", r['data']['modhash'])
    print("dist: ", r['data']['dist'])
    print("before: ", r['data']['before'])
    print("after: ", r['data']['after'])


    #//*** Keys to exclude from display while fishing for relevant displays
    exclude_keys = ['approved_at_utc',
                    'saved',
                    'mod_reason_title',
                    'clicked',
                    'link_flair_richtext',
                    'hidden',
                    'top_awarded_type',
                    'hide_score',
                    'quarantine',
                    'upvote_ratio',
                    'author_flair_background_color',
                    'media_embed',
                    'author_flair_template_id',
                    'secure_media',
                    'link_flair_text',
                    'removed_by_category',
                    'can_mod_post',
                    'approved_by',
                    'edited',
                    'author_flair_css_class',
                    'author_flair_richtext',
                    'thumbnail',
                    'gildings',
                    'all_awardings',
                    'is_created_from_ads_ui',
                    'link_flair_type',
                    'author_flair_type',
                    'domain',
                    'selftext_html',
                    'banned_at_utc',
                    'link_flair_template_id',
                    'spoiler',
                    'over_18',
                    'visited',
                    'is_reddit_media_domain',
                    'awarders',
                    'treatment_tags',
                    'removed_by',
                    'num_reports',
                    'mod_reason_by',
                    'removal_reason',
                    'link_flair_background_color',
                    'report_reasons',
                    'discussion_type',
                    'send_replies',
                    'mod_reports',
                    'author_patreon_flair',
                    'author_flair_text_color',
                    'parent_whitelist_status',
                    'stickied',
                    'link_flair_css_class' ] 
    
    children = r["data"]["children"]
    for child in children:
        print("===============")
        print("===============")
        print("===============")

        #//*** Print Some data for reference
        post = child['data']
        print(post['permalink'])
        print("Title: ",post['title'])
        print(datetime.fromtimestamp(post['created_utc']))
        print(f"Comments: {post['num_comments']} Ups: {post['ups']} Downs: {post['downs']} Cross posts:  {post['num_crossposts']}")
        print(post['url'])
    #//*** This code displays
    #for key,value in child['data'].items():
    #    if key not in exclude_keys:
#        if key not in ['all_awardings']:
    #        print(key," - " , value)
 #https://www.reddit.com/r/wallstreetbets/comments/ns4bqz/amc_ceo_shows_us_he_isnt_wearing_pants.json   
    #time.sleep(.001)
    
    #//*** Return After. The next call should be after if not None
    return r['data']['after']

In [22]:
print(sub_df)

Empty DataFrame
Columns: [date, up, down, cross, link, title]
Index: []
